# Connecting to the Playground API
This Notebook shows how to connect to the Intelligence Playground API and do amazing stuff!

## Some standard imports

In [1]:
# Standard imports
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np
from skimage import exposure, color, img_as_ubyte
import mercantile
import requests
import csv
import uuid
import json
import geohash2
import uuid
import warnings
import geojson
import time
warnings.filterwarnings("ignore")

from shapely.geometry import mapping, shape

# Widgets imports
from ipyleaflet import Map, TileLayer, Polygon
import ipywidgets as widgets
from IPython.core.display import HTML

## Connecting to the API

To connect to the API, you need to retrieve your API_KEYS from the OneAtlas website. Follow the simple steps below:

1. Visit this URL: https://data.api.oneatlas.airbus.com/api-keys
2. Click the **Create and API key** button
3. Enter a description for your API_KEY (e.g. Playground Keys)
4. Store the file in the same folder than this notebook and name it **api_key.txt**

Make sure to keep your **api_key.txt** safe! Do not include it in a public github repository for example :-)

The following script will then use this **api_key.txt** file to generate an ACCESS_TOKEN. We will store this ACCESS_TOKEN in HEADERS that we will send with each requests. The ACCESS_TOKEN has a timeout so we will create a function that renew the ACCESS_TOKEN when half of the timeout has expired. 

In [2]:
# TOKENS and OAUTH
ACCESS_TOKEN = None
TIMEOUT = None
HEADERS = None

# read API key
with open('api_key.txt') as f:
    api_key = f.readline()

#FUNCTIONS
def playground_refresh_access_token():
    global ACCESS_TOKEN, TIMEOUT, HEADERS
    
    # if ACCESS_TOKEN exists and timeout is not reached, HEADERS should be OK
    if ACCESS_TOKEN is not None:
        if TIMEOUT is not None:
            if time.time() < TIMEOUT:
                return True
            
    # request auth token
    r = requests.post('https://authenticate.foundation.api.oneatlas.airbus.com/auth/realms/IDP/protocol/openid-connect/token',
        headers={'Content-Type':'application/x-www-form-urlencoded'},
        data={'apikey':api_key, 'grant_type':'api_key', 'client_id':'IDP'})

    # Check status code
    assert r.status_code == 200, 'A problem occured during connection \
                                  with the Playground'
    # Convert content in json
    content = r.json()
    
    assert 'access_token' in content.keys(), 'No access_token field in response'
    ACCESS_TOKEN = content['access_token']
    assert 'expires_in' in content.keys(), 'No expires_in field in response'
    expires_in = content['expires_in']
    TIMEOUT = time.time() + expires_in // 2

    # build headers
    HEADERS = {
        'Authorization': 'Bearer {}'.format(ACCESS_TOKEN),
        'Content-Type': 'application/json',
        'Cache-Control': 'no-cache'
    }
    return True

if playground_refresh_access_token():
    print("Successfully connected to Playground.")


Successfully connected to Playground.


## Library of useful functions

In [7]:
# Official Playground URL
#PLAYGROUND_URL = "https://playground.intelligence-airbusds.com"

# Alternate Playground URL
PLAYGROUND_URL = "https://apps.playground.airbusds-geo.com"

PLAYGROUND_CURRENT_USER_URL = PLAYGROUND_URL + "/currentUser"
def check_logged_user():
    # refresh the Playground token if needed
    playground_refresh_access_token()
    # ping current user
    r = requests.get(PLAYGROUND_CURRENT_USER_URL, headers=HEADERS)
    # Check status
    assert r.status_code == 200, 'A problem occured during connection \
                                  with the Playground'
    # Return result
    return r.json()

PLAYGROUND_PROJECT_URL = PLAYGROUND_URL + "/api/projects"
def get_projects():
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_PROJECT_URL, headers=HEADERS)
    #print(r.status_code)
    #print(r.content)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

PLAYGROUND_PROCESSES_URL = PLAYGROUND_URL + "/api/v1/processes?projectId={projectId}"
def get_processes(projectId):
    #print("Retrieving information for zone with id={}".format(zone_id))
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_PROCESSES_URL.format(projectId=projectId), headers=HEADERS)
    #print(r.status_code)
    #print(r.content)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

PLAYGROUND_PROCESS_URL = PLAYGROUND_URL + "/api/v1/processes/{processId}?projectId={projectId}"
def get_process(processId):
    #print("Retrieving information for zone with id={}".format(zone_id))
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_PROCESS_URL.format(projectId=projectId, processId=processId), headers=HEADERS)
    #print(r.status_code)
    #print(r.content)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

PLAYGROUND_DATASETS_URL = PLAYGROUND_URL + "/api/v1/datasets?projectId={projectId}"
def get_datasets(projectId):
    #print("Retrieving information for zone with id={}".format(zone_id))
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_DATASETS_URL.format(projectId=projectId), headers=HEADERS)
    #print(r.status_code)
    #print(r.content)
    if r.status_code == 404:
        print('This project ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

PLAYGROUND_DATASET_URL = PLAYGROUND_URL + "/api/datasets/{datasetId}"
def get_dataset(dataset_id):
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_DATASET_URL.format(datasetId=dataset_id), headers=HEADERS)
    #print(r.status_code)
    if r.status_code == 404:
        print('This dataset ID does not exist')
    if r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

PLAYGROUND_ZONES_SEARCH_URL = PLAYGROUND_URL + "/api/zones?dataset_id={datasetId}"
def get_zones_in_dataset(dataset_id):
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_ZONES_SEARCH_URL.format(datasetId=dataset_id), headers=HEADERS)
    #print(r.status_code)
    if r.status_code == 404:
        print('This dataset ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

PLAYGROUND_ZONE_URL = PLAYGROUND_URL + "/api/zones/{zoneId}"
def get_zone(zone_id):
    print("Retrieving information for zone with id={}".format(zone_id))
    playground_refresh_access_token()
    r = requests.get(PLAYGROUND_ZONE_URL.format(zoneId=zone_id), headers=HEADERS)
    #print(r.status_code)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

def store_zone(zone_id, zone):
    print("Updating information for zone with id={}".format(zone_id))
    playground_refresh_access_token()
    r = requests.put(PLAYGROUND_ZONE_URL.format(zoneId=zone_id), headers=HEADERS, data=json.dumps(zone))
    #print(r.status_code)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

RECORDS_COUNT_ZONE_URL = PLAYGROUND_URL + "/api/records?count=true&zone_id={zoneId}&bbox={BBOX}"
def get_records_count_in_zone(zone_id, bbox):
    playground_refresh_access_token()
    r = requests.get(RECORDS_COUNT_ZONE_URL.format(zoneId=zone_id, BBOX=bbox), headers=HEADERS)
    #print(r.status_code)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

RECORDS_ZONE_URL = PLAYGROUND_URL + "/api/records?zone_id={zoneId}&bbox={BBOX}"
def get_records_in_zone(zone_id, bbox):
    playground_refresh_access_token()
    r = requests.get(RECORDS_ZONE_URL.format(zoneId=zone_id, BBOX=bbox), headers=HEADERS)
    #print(r.status_code)
    if r.status_code == 404:
        print('This zone ID does not exist')
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return r.json()
    return None

RECORDS_URL = PLAYGROUND_URL + "/api/records/{recordId}"
def delete_record(record_id):
    playground_refresh_access_token()
    r = requests.delete(RECORDS_URL.format(recordId=record_id), headers=HEADERS)
    #print(r.status_code)
    if r.status_code == 404:
        print('No record with ID={} found in database.'.format(record_id))
    elif r.status_code == 401:
        print('You do not have sufficient rights to perform this operation')
    elif r.status_code != 200:
        print("A problem occured during connection with the Playground (status_code={})".format(r.status_code))
    else:
        return True
    return False

In [8]:
def print_json(parsed):
    print(json.dumps(parsed, indent=4, sort_keys=True))
    
def getBounds(coords):
    xmax = ymax = -np.Infinity
    xmin = ymin = np.Infinity
    for v in coords:
        xmax = xmax if xmax > v[0] else v[0]
        ymax = ymax if ymax > v[1] else v[1]
        xmin = xmin if xmin < v[0] else v[0]
        ymin = ymin if ymin < v[1] else v[1]
    return (xmin, ymin, xmax, ymax)


## Now start using them!

In [9]:
# Logged in user
user = check_logged_user()
print("Logged as user: {} {}".format(user['firstname'], user['lastname']))

Logged as user: Jean-Francois Faudi


## Get available projects for this user

In [10]:
p_database = {}
p_label = []
r = get_projects()
#print(r)
for project in r:
    #print(process)
    project_id = project['project_id']
    p_label.append((project['name'], project_id))
    p_database[project_id] = project
    
p_label.sort() 
p_w = widgets.Dropdown(
    options=p_label,
    description='Project:',
    disabled=False
)
display(p_w)

Dropdown(description='Project:', options=(('Airbus', '1a50823a-d9c1-4397-8b64-1c8079d6c7eb'), ('Airbus NA', 'e…

## Get available processes in this project

In [11]:
projectId = p_w.value

a_database = {}
a_label = []
r = get_processes(projectId)
#print(processes)
for process in r['processes']:
    #print(process)
    process_id = process['id']
    a_label.append((process['title'], process_id))
    a_database[process_id] = process
    
a_label.sort() 
a_w = widgets.Dropdown(
    options=a_label,
    description='Process:',
    disabled=False
)
display(a_w)

Dropdown(description='Process:', options=(('DL Change Detection 5', '2f8d2fc7-e38e-4e2f-88f2-266135677f4c'), (…

In [12]:
process_id = a_w.value
print("Process_ID={}".format(process_id))
r = get_process(process_id)
#print_json(r)
print_json(r['input'])

Process_ID=2f8d2fc7-e38e-4e2f-88f2-266135677f4c
{
    "$schema": "http://json-schema.org/draft-06/schema#",
    "description": "Geo Processes Manager API input schema for tile change detection",
    "oneOf": [
        {
            "required": [
                "datasetId"
            ]
        },
        {
            "required": [
                "datasetName"
            ]
        }
    ],
    "properties": {
        "datasetId": {
            "description": "Result dataset ID, if the dataset already exists (exclusive with datasetName)",
            "type": "string"
        },
        "datasetName": {
            "description": "Result dataset name, to create a new dataset (exclusive with datasetId)",
            "type": "string"
        },
        "geom": {
            "description": "Area of interest",
            "properties": {
                "coordinates": {
                    "description": "GeoJSON Polygon coordinates",
                    "type": "array"
                },

In [13]:
print_json(r['_links'])

{
    "delete": {
        "href": "https://playground-api-gateway.playground.airbusds-geo.com/api/v1/processes/2f8d2fc7-e38e-4e2f-88f2-266135677f4c?projectId=3361252e-bfe8-4b8e-8c38-e915e32d741f",
        "method": "DELETE",
        "relation": "Delete the process",
        "type": "application/json"
    },
    "jobs": {
        "href": "https://playground-api-gateway.playground.airbusds-geo.com/api/v1/processes/2f8d2fc7-e38e-4e2f-88f2-266135677f4c/jobs?projectId=3361252e-bfe8-4b8e-8c38-e915e32d741f",
        "method": "GET",
        "relation": "Return the list of all process' jobs",
        "type": "application/json"
    },
    "list": {
        "href": "https://playground-api-gateway.playground.airbusds-geo.com/api/v1/processes?projectId=3361252e-bfe8-4b8e-8c38-e915e32d741f",
        "method": "GET",
        "relation": "Return the list of all processes",
        "type": "application/json"
    },
    "runNewJob": {
        "href": "https://playground-api-gateway.playground.airbusds-

In [14]:
# TODO: this field should contain information about the output format of the process
print_json(r['output'])

null


## Get available datasets in this project

In [15]:
projectId = p_w.value

d_database = {}
d_label = []
r = get_datasets(projectId)
#print_json(r)
for dataset in r['datasets']:
    #print_json(dataset)
    dataset_id = dataset['datasetId']
    d_label.append((dataset['name'], dataset_id))
    d_database[dataset_id] = dataset
    
d_label.sort() 
d_w = widgets.Dropdown(
    options=d_label,
    description='Dataset:',
    disabled=False
)
display(d_w)

Dropdown(description='Dataset:', options=(('Buildings', 'ba61290c-2ebc-4dfe-8c89-2700d2251be8'), ('Cnes', 'c6f…

In [16]:
# Dataset
dataset_id = d_w.value

# display information about this dataset
r = get_dataset(dataset_id)
#print_json(r)
print("This dataset contains {} zones.".format(r['zones']))
print("This dataset contains {} records.".format(r['records']))

This dataset contains 1 zones.
This dataset contains 9608 records.


In [17]:
z_database = {}
z_label = []
zones = get_zones_in_dataset(dataset_id)
for zone in zones:
    zone_id = zone['zone_id']
    z_label.append((zone['name'], zone_id))
    z_database[zone_id] = zone
    
z_label.sort() 
z_w = widgets.Dropdown(
    options=z_label,
    description='Zones:',
    disabled=False
)
display(z_w)

Dropdown(description='Zones:', options=(('job_2019-05-20_Denver Buildings', '532c8037-bdad-4f2f-b3c8-63ff78fa4…

In [19]:
zone_id = z_w.value
print("Zone_ID={}".format(zone_id))
#print_json(z_database[zone_id])
print_json(z_database[zone_id]['geom']['coordinates'][0])

Zone_ID=532c8037-bdad-4f2f-b3c8-63ff78fa4d8e
[
    [
        -105.008674,
        39.669472
    ],
    [
        -104.935118,
        39.669472
    ],
    [
        -104.935118,
        39.68962
    ],
    [
        -105.008674,
        39.68962
    ],
    [
        -105.008674,
        39.669472
    ]
]


In [21]:
bounds = getBounds(z_database[zone_id]['geom']['coordinates'][0])
bbox = ",".join(map(str, bounds))
r = get_records_count_in_zone(zone_id, bbox)
#print_json(r)
print("This zone contains {} records.".format(r['count']))

This zone contains 9580 records.
